In [0]:
#!/usr/bin/env python3
"""
main.py - Root entry point that imports and executes create_view.py from same directory
GitHub Repo Structure:
project/
├── main.py                 # This file (entry point for Databricks Jobs)
└── create_view.py          # Your notebook code saved as .py
"""
import sys
import os
import argparse
from pyspark.sql import SparkSession
import logging
import importlib.util

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger("view_creator_main")

def import_create_view_module():
    """Import create_view.py from same directory as main.py"""
    script_dir = os.path.dirname(os.path.abspath(__file__))
    create_view_path = os.path.join(script_dir, "create_view.py")
    
    if not os.path.exists(create_view_path):
        print(f"❌ create_view.py not found at: {create_view_path}")
        print("📁 Expected GitHub structure:")
        print("   project/")
        print("   ├── main.py")
        print("   └── create_view.py")
        sys.exit(1)
    
    # Add current directory to Python path
    if script_dir not in sys.path:
        sys.path.insert(0, script_dir)
    
    # Import create_view module
    spec = importlib.util.spec_from_file_location("create_view", create_view_path)
    create_view_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(create_view_module)
    
    return create_view_module

def main():
    """Parse CLI arguments and execute create_view.run()"""
    parser = argparse.ArgumentParser(
        description="Create Unity Catalog VIEW - Calls create_view.py",
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
GitHub Workflow Setup:
1. Repo structure: main.py | create_view.py
2. Git integration: Workspace > Repos > Add repo
3. Job config:
   Task Type: Spark Python
   File: /Repos/{username}/{repo}/{branch}/main.py
   Parameters: dataeng dataeng source_table target_view create false

Examples:
  dataeng dataeng schema_evo_merge_schema_t3 schema_evo_merge_schema_demo9 create false
        """
    )
    parser.add_argument("catalog_name", nargs="?", default="dataeng")
    parser.add_argument("schema_name", nargs="?", default="dataeng")
    parser.add_argument("source_table_name", nargs="?", default="schema_evo_merge_schema_t3")
    parser.add_argument("target_view_name", nargs="?", default="schema_evo_merge_schema_demo9")
    parser.add_argument("operation", nargs="?", default="create", choices=["create", "alter"])
    parser.add_argument("dryrun", nargs="?", type=str, default="false")
    
    args = parser.parse_args()
    dryrun = args.dryrun.lower() in ("true", "1", "yes", "y")
    
    # Validate Spark session
    try:
        spark = SparkSession.getActiveSession()
        if spark is None:
            print("❌ No Spark session. Run in Databricks.")
            sys.exit(1)
    except:
        print("❌ Spark session unavailable")
        sys.exit(1)
    
    # Import and execute create_view.py
    print("🚀 Importing create_view.py...")
    try:
        create_view_module = import_create_view_module()
        print("✅ create_view.py loaded successfully")
        
        print(f"📋 Executing: {args.catalog_name}.{args.schema_name}.{args.target_view_name}")
        print(f"   Source: {args.catalog_name}.{args.schema_name}.{args.source_table_name}")
        print(f"   Mode: {args.operation} ({'dryrun' if dryrun else 'live'})")
        
        # Call the run() function from your notebook
        create_view_module.run(
            spark=spark,
            catalog_name=args.catalog_name,
            schema_name=args.schema_name,
            source_table_name=args.source_table_name,
            target_view_name=args.target_view_name,
            operation=args.operation,
            dryrun=dryrun
        )
        
        print("🎉 Job completed successfully!")
        
    except Exception as e:
        print(f"❌ Execution failed: {e}")
        logger.error(f"Error: {e}", exc_info=True)
        sys.exit(1)

if __name__ == "__main__":
    main()

In [0]:
%run /FileStore/project/main.py $dryrun="true" 